## Aggregate Function Example
> Time-series aggregation function   
  performs a rolling aggregation on {df_artifact}, over {window} by the selected {keys}
  applying {metric_aggs} on {metrics} and {label_aggs} on {labels}. adding {suffix} to the 
  feature names.
    
    

In [ ]:
# Make sure mlrun is aligned !
# !/User/align_mlrun.sh

In [14]:
import pandas as pd
import mlrun
mlrun.set_environment(project='function-marketplace')

> 2021-10-05 09:16:39,169 [info] loaded project function-marketplace from MLRun DB


('function-marketplace', 'v3io:///projects/{{run.project}}/artifacts')

In [23]:
# Getting & inspecting original dataset
DATA = "https://s3.wasabisys.com/iguazio/data/market-palce/aggregate/metrics.pq"
original = pd.read_parquet(DATA)
original.head().style.set_caption("Original Dataframe !")

In [24]:
# import the function
aggregate_function = mlrun.import_function("hub://aggregate").apply(mlrun.auto_mount())

In [25]:
aggregate_run = aggregate_function.run(params={'metrics': ['cpu_utilization'],
               'labels': ['is_error'],
               'metric_aggs': ['mean', 'sum'],
               'label_aggs': ['max'],
               'suffix': 'daily',
               'inplace': False,
               'window': 5,
               'center': True,               
               'files_to_select': 2}           
           , inputs={'df_artifact': DATA}
           )
    

> 2021-10-05 09:22:15,709 [info] starting run aggregate-aggregate uid=7b2d8ccfca8c4cdb8f2ddf8389bc3843 DB=http://mlrun-api:8080
> 2021-10-05 09:22:15,855 [info] Job is running in the background, pod: aggregate-aggregate-s22gh
> 2021-10-05 09:22:21,445 [info] Aggregating https://s3.wasabisys.com/iguazio/data/market-palce/aggregate/metrics.pq
> 2021-10-05 09:22:22,776 [info] Logging artifact
> 2021-10-05 09:22:22,945 [info] run executed, status=completed
Converting input from bool to <class 'numpy.uint8'> for compatibility.
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
function-marketplace,...89bc3843,0,Oct 05 09:22:21,completed,aggregate-aggregate,v3io_user=danikind=jobowner=danihost=aggregate-aggregate-s22gh,df_artifact,"metrics=['cpu_utilization']labels=['is_error']metric_aggs=['mean', 'sum']label_aggs=['max']suffix=dailyinplace=Falsewindow=5center=Truefiles_to_select=2",,aggregate


> 2021-10-05 09:22:25,063 [info] run executed, status=completed


Convert the code to an MLRun function

### Show results

In [29]:
aggregate_run.artifact('aggregate').as_df().head().style.set_caption("Aggregated Dataframe !")